In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("hw7.ipynb")

# CPSC 330 - Applied Machine Learning 

## Homework 7: Word embeddings and topic modeling 
**Due date: See the [Calendar](https://htmlpreview.github.io/?https://github.com/UBC-CS/cpsc330/blob/master/docs/calendar.html).**

## Imports

In [2]:
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.metrics import classification_report

<br><br>

## Submission instructions
<hr>
rubric={points:2}

You will receive marks for correctly submitting this assignment. To submit this assignment, follow the instructions below:

- **You may work on this assignment in a group (group size <= 4) and submit your assignment as a group.** 
- Below are some instructions on working as a group.  
    - The maximum group size is 4. 
    - You can choose your own group members. 
    - Use group work as an opportunity to collaborate and learn new things from each other. 
    - Be respectful to each other and make sure you understand all the concepts in the assignment well. 
    - It's your responsibility to make sure that the assignment is submitted by one of the group members before the deadline. [Here](https://help.gradescope.com/article/m5qz2xsnjy-student-add-group-members) are some instructions on adding group members in Gradescope.  
- Upload the .ipynb file to Gradescope.
- **If the .ipynb file is too big or doesn't render on Gradescope for some reason, also upload a pdf or html in addition to the .ipynb.** 
- Make sure that your plots/output are rendered properly in Gradescope.

<br><br><br><br>

## Exercise 1:  Exploring pre-trained word embeddings <a name="1"></a>
<hr>

In lecture 18, we talked about natural language processing (NLP). Using pre-trained word embeddings is very common in NLP. It has been shown that pre-trained word embeddings work well on a variety of text classification tasks. These embeddings are created by training a model like Word2Vec on a huge corpus of text such as a dump of Wikipedia or a dump of the web crawl. 

A number of pre-trained word embeddings are available out there. Some popular ones are: 

- [GloVe](https://nlp.stanford.edu/projects/glove/)
    * trained using [the GloVe algorithm](https://nlp.stanford.edu/pubs/glove.pdf) 
    * published by Stanford University 
- [fastText pre-trained embeddings for 294 languages](https://fasttext.cc/docs/en/pretrained-vectors.html) 
    * trained using the fastText algorithm
    * published by Facebook
    
In this exercise, you will be exploring GloVe Wikipedia pre-trained embeddings. The code below loads the word vectors trained on Wikipedia using an algorithm called Glove. You'll need `gensim` package in your cpsc330 conda environment to run the code below. 

```
> conda activate cpsc330
> conda install -c anaconda gensim
```

In [3]:
import gensim
import gensim.downloader

print(list(gensim.downloader.info()["models"].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [4]:
# This will take a while to run when you run it for the first time.
import gensim.downloader as api

glove_wiki_vectors = api.load("glove-wiki-gigaword-100")

In [5]:
len(glove_wiki_vectors)

400000

There are 400,000 word vectors in this pre-trained model. 

Now that we have GloVe Wiki vectors loaded in `glove_wiki_vectors`, let's explore the embeddings. 

<br><br>

<!-- BEGIN QUESTION -->

### 1.1 Word similarity using pre-trained embeddings
rubric={points:2}

**Your tasks:**

- Come up with a list of 4 words of your choice and find similar words to these words in `glove_wiki_vectors` embeddings. 

<div class="alert alert-warning">

Solution_1.1
    
</div>

In [6]:
word_list=["tree","happy","sky","park"]

In [7]:
print(glove_wiki_vectors.most_similar("tree"))

[('trees', 0.8510393500328064), ('grass', 0.6922113299369812), ('pine', 0.6856961846351624), ('oak', 0.6387961506843567), ('leaf', 0.6364573836326599), ('bushes', 0.6336526870727539), ('flower', 0.6273626089096069), ('green', 0.6112821102142334), ('garden', 0.603394091129303), ('bark', 0.6031671762466431)]


In [8]:
print(glove_wiki_vectors.most_similar("happy"))

[("'m", 0.8413287997245789), ('feel', 0.8132575750350952), ("'re", 0.8048083186149597), ('i', 0.7938276529312134), ("'ll", 0.7916273474693298), ('really', 0.7903971076011658), ('glad', 0.7833361029624939), ('good', 0.7821646332740784), ('we', 0.7808917164802551), ('sure', 0.7788466215133667)]


In [9]:
print(glove_wiki_vectors.most_similar("sky"))

[('skies', 0.7078207731246948), ('light', 0.6485515832901001), ('bright', 0.633054792881012), ('blue', 0.6260656714439392), ('dark', 0.6218482255935669), ('cloud', 0.6198915243148804), ('horizon', 0.6129894256591797), ('sun', 0.6126757860183716), ('lights', 0.6123525500297546), ('darkness', 0.6013585925102234)]


In [10]:
print(glove_wiki_vectors.most_similar("park"))

[('parks', 0.7290407419204712), ('riverside', 0.6925320625305176), ('hill', 0.6913207769393921), ('recreation', 0.6909290552139282), ('forest', 0.6692465543746948), ('garden', 0.667052149772644), ('hall', 0.6595367193222046), ('gardens', 0.6573269367218018), ('road', 0.6527218222618103), ('located', 0.6472485065460205)]


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.2 Word similarity using pre-trained embeddings
rubric={points:2}

**Your tasks:**
1. Calculate cosine similarity for the following word pairs (`word_pairs`) using the [`similarity`](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) method of `glove_wiki_vectors`.

In [11]:
word_pairs = [
    ("coast", "shore"),
    ("clothes", "closet"),
    ("old", "new"),
    ("smart", "intelligent"),
    ("dog", "cat"),
    ("tree", "lawyer"),
]

<div class="alert alert-warning">

Solution_1.2
    
</div>

In [12]:
# From lecture 18 code

for word in word_pairs:
    print("The similarity between " + word[0] + " and " + word[1] + " is " + str(glove_wiki_vectors.similarity(word[0], word[1])))

The similarity between coast and shore is 0.70002717
The similarity between clothes and closet is 0.54627603
The similarity between old and new is 0.6432488
The similarity between smart and intelligent is 0.7552733
The similarity between dog and cat is 0.8798075
The similarity between tree and lawyer is 0.076719455


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.3 Stereotypes and biases in embeddings
rubric={points:6}

Word vectors contain lots of useful information. But they also contain stereotypes and biases of the texts they were trained on. In the lecture, we saw an example of gender bias in Google News word embeddings. Here we are using pre-trained embeddings trained on Wikipedia data. 

**Your tasks:**

1. Explore whether there are any worrisome biases or stereotypes present in these embeddings or not by trying out at least 4 examples. You can use the following two methods or other methods of your choice to explore what kind of stereotypes and biases are encoded in these embeddings. 
    - the `analogy` function below which gives word analogies (an example shown below)
    - [similarity](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) or [distance](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=distance#gensim.models.keyedvectors.KeyedVectors.distances) methods (an example is shown below)   
2. Discuss your observations.

> Note that most of the recent embeddings are de-biased. But you might still observe some biases in them. Also, not all stereotypes present in pre-trained embeddings are necessarily bad. But you should be aware of them when you use them in your models. 

An example of using analogy to explore biases and stereotypes.  

In [13]:
def analogy(word1, word2, word3, model=glove_wiki_vectors):
    """
    Returns analogy word using the given model.

    Parameters
    --------------
    word1 : (str)
        word1 in the analogy relation
    word2 : (str)
        word2 in the analogy relation
    word3 : (str)
        word3 in the analogy relation
    model :
        word embedding model

    Returns
    ---------------
        pd.dataframe
    """
    print("%s : %s :: %s : ?" % (word1, word2, word3))
    sim_words = model.most_similar(positive=[word3, word2], negative=[word1])
    return pd.DataFrame(sim_words, columns=["Analogy word", "Score"])

In [14]:
analogy("man", "doctor", "woman")

man : doctor :: woman : ?


,Analogy word,Score
0,nurse,0.773523
1,physician,0.718943
2,doctors,0.682433
3,patient,0.675068
4,dentist,0.672603
5,pregnant,0.664246
6,medical,0.652045
7,nursing,0.645348
8,mother,0.639333
9,hospital,0.638750


An example of using similarity between words to explore biases and stereotypes.  

In [15]:
glove_wiki_vectors.similarity("white", "rich")

0.44723597

In [16]:
glove_wiki_vectors.similarity("black", "rich")

0.51745194

<div class="alert alert-warning">

Solution_1.3
    
</div>

_Type your answer here, replacing this text._

In [17]:
glove_wiki_vectors.similarity("mexican", "drug")

0.49212295

In [18]:
analogy("italian", "cheese", "korean")

italian : cheese :: korean : ?


,Analogy word,Score
0,egg,0.572073
1,peanut,0.546498
2,meat,0.541079
3,kimchi,0.539273
4,butter,0.524613
5,chicken,0.516982
6,beef,0.516147
7,dumplings,0.513282
8,korea,0.509815
9,crackers,0.506846


In [19]:
analogy("french", "baguette", "indian")

french : baguette :: indian : ?


,Analogy word,Score
0,flatbread,0.555339
1,pita,0.525241
2,paintbrush,0.496433
3,hopi,0.495832
4,granny,0.493582
5,peeled,0.485426
6,unpeeled,0.478829
7,crosswise,0.473535
8,clove,0.472905
9,blackfeet,0.469275


In [20]:
analogy("man", "successful", "woman")

man : successful :: woman : ?


,Analogy word,Score
0,unsuccessful,0.676052
1,success,0.628026
2,promising,0.616110
3,becoming,0.598911
4,pioneering,0.573730
5,first,0.572089
6,become,0.571651
7,best,0.560237
8,women,0.558045
9,ambitious,0.556603


The above similarity/analogy exercise gave some interesting insights into biases. 
For example, there is a very high similarity between Mexican and Drugs, which is a common stereotype which is indeed quite a harmful one. 
One of the analogy based on man is to successful gives woman the highest similarity to unsuccessful. This shows the highly misogynystic bias in this embedding that is very harmful as well. 
These examples show that even after debiasing the embeddings, there is still a strong level of harmful stereotypes/biases present. 

Some of the more funny ones were the analogy between french is to baguette while an indian is to flatbread as well as italian to cheese and korean with a similarity to kimchi and dumplings. The embedding reflected some of the cultural connections to food and different people which was quite interesting. Even though its not a harmful stereotype, we should be cautious since often times food is essential to our culture and certain foods come with their misconceptions that can then lead to racial biases. 


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.4 Discussion
rubric={points:4}

**Your tasks:**
1. Based on your exploration above, comment on the overall quality of these pre-trained embeddings. 
2. In the lecture, we saw that our pre-trained word embedding model output an analogy that reinforced a gender stereotype. Give an example of how using such a model could cause harm in the real world.

<div class="alert alert-warning">

Solution_1.4
    
</div>

1. In terms of quality of the embeddings, from our example exploring that indian is analogous to flat bread if French is analogous to Baguette shows that the embedding is being able to pick up strong similarity like these. However not all similarity is good if they represent stereotypes and biases that can harm the society. When an embedding like this is deployed in essential things like court, healthcare, high similarity between words should not be the main thing to look out for, instead it needs to reflect a just system. Hence, overall even though for most examples the quality of these pre-trained embeddings is quite good, it needs to undergo a lot more de-biasing to be a high quality non-stereotypical embedding.

2. A model that outputs gender stereotype reinforcing analogy can be harmful in the real world in many ways. For example, if students were to search up career options and the information they receive was man to programmer is like woman to homemaking, female students could potentially be discourage to pursue in the STEM related field. Similarly, an aspiring nursing male could have easily find the result that the similarity between the word "man" and "nurse" is way lower that of the female counterpart. These are both examples of why pre-trained word embedding models should be carefully handled and debiased.

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.5 Classification with pre-trained embeddings 
rubric={points:8}

In lecture 18, we saw that you can conveniently get word vectors with `spaCy` with `en_core_web_md` model. In this exercise, you'll use word embeddings in multi-class text classification task. We will use [HappyDB](https://www.kaggle.com/ritresearch/happydb) corpus which contains about 100,000 happy moments classified into 7 categories: *affection, exercise, bonding, nature, leisure, achievement, enjoy_the_moment*. The data was crowd-sourced via [Amazon Mechanical Turk](https://www.mturk.com/). The ground truth label is not available for all examples, and in this lab, we'll only use the examples where ground truth is available (~15,000 examples). 

- Download the data from [here](https://www.kaggle.com/ritresearch/happydb).
- Unzip the file and copy it under data/ directory in this homework directory.

The code below reads the data CSV (assuming that it's present in the current directory as *cleaned_hm.csv*),  cleans it up a bit, and splits it into train and test splits. 

**Your tasks:**

1. Train logistic regression with bag-of-words features (`CountVectorizer`) and show [classification report](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html) on the test set. 
2. Train logistic regression with average embedding representation extracted using spaCy and show [classification report](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html) on the test set. (You can refer to lecture 18 notes for this.)  

In [21]:
df = pd.read_csv("data/cleaned_hm.csv", index_col=0)
sample_df = df.dropna()
sample_df.head()

,wid,reflection_period,original_hm,cleaned_hm,modified,num_sentence,ground_truth_category,predicted_category
hmid,,,,,,,,
27676,206,24h,We had a serious talk with some friends of our...,We had a serious talk with some friends of our...,True,2,bonding,bonding
27678,45,24h,I meditated last night.,I meditated last night.,True,1,leisure,leisure
27697,498,24h,My grandmother start to walk from the bed afte...,My grandmother start to walk from the bed afte...,True,1,affection,affection
27705,5732,24h,I picked my daughter up from the airport and w...,I picked my daughter up from the airport and w...,True,1,bonding,affection
27715,2272,24h,when i received flowers from my best friend,when i received flowers from my best friend,True,1,bonding,bonding


In [22]:
sample_df = sample_df.rename(
    columns={"cleaned_hm": "moment", "ground_truth_category": "target"}
)

In [23]:
train_df, test_df = train_test_split(sample_df, test_size=0.3, random_state=123)
X_train, y_train = train_df["moment"], train_df["target"]
X_test, y_test = test_df["moment"], test_df["target"]

You need `spacy` to run the code below. If it's not in your course conda environment, you need to install it. 

> conda install -c conda-forge spacy

You also need to download the following language model. 

> python -m spacy download en_core_web_md

In [24]:
import spacy

nlp = spacy.load("en_core_web_md")

<div class="alert alert-warning">

Solution_1.5
    
</div>

_Type your answer here, replacing this text._

In [25]:
# From lecture 18 and scikit learn manual 

# 1. Training Logistic Regression and CountVectorizer
pipe = make_pipeline(CountVectorizer(stop_words="english"), LogisticRegression(max_iter=8000))
pipe.fit(X_train, y_train)

y_pred = pipe.predict(X_test)

print(classification_report(y_test, y_pred))

print("Train accuracy {:.2f}".format(pipe.score(X_train, y_train)))
print("Test accuracy {:.2f}".format(pipe.score(X_test, y_test)))

                  precision    recall  f1-score   support

     achievement       0.79      0.87      0.83      1302
       affection       0.90      0.91      0.91      1423
         bonding       0.91      0.85      0.88       492
enjoy_the_moment       0.60      0.54      0.57       469
        exercise       0.91      0.57      0.70        74
         leisure       0.73      0.70      0.71       407
          nature       0.73      0.46      0.57        71

        accuracy                           0.82      4238
       macro avg       0.80      0.70      0.74      4238
    weighted avg       0.82      0.82      0.81      4238

Train accuracy 0.96
Test accuracy 0.82


In [26]:
# 2. From Lecture 18 

# Average embedding and CountVectorizer

X_train_embeddings = pd.DataFrame([text.vector for text in nlp.pipe(X_train)])
X_test_embeddings = pd.DataFrame([text.vector for text in nlp.pipe(X_test)])
regression = LogisticRegression(max_iter=8000)
regression.fit(X_train_embeddings, y_train)
y_pred=regression.predict(X_test_embeddings)
print(classification_report(y_test, y_pred))

                  precision    recall  f1-score   support

     achievement       0.81      0.83      0.82      1302
       affection       0.86      0.91      0.89      1423
         bonding       0.83      0.77      0.80       492
enjoy_the_moment       0.57      0.51      0.54       469
        exercise       0.68      0.76      0.72        74
         leisure       0.72      0.65      0.68       407
          nature       0.69      0.72      0.70        71

        accuracy                           0.79      4238
       macro avg       0.74      0.74      0.74      4238
    weighted avg       0.79      0.79      0.79      4238



In [27]:
print("Train accuracy {:.2f}".format(regression.score(X_train_embeddings, y_train)))
print("Test accuracy {:.2f}".format(regression.score(X_test_embeddings, y_test)))

Train accuracy 0.85
Test accuracy 0.79


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.6 Discussion
rubric={points:6}

**Your tasks:**
1. Briefly explain the difference between using `CountVectorizer` vs. average-embedding approach for text classification.  
2. Which representation among these two would be more interpretable? Why?   
3. Are we using any transfer learning here? If yes, are you observing any benefits of transfer learning? Briefly discuss. 

<div class="alert alert-warning">

Solution_1.6
    
</div>

1. CountVectorizer creates a matrix of word vectors and counts how many times the words occurred. 
Average embedding is an approach to use word embeddings which contains word representations with similar words grouped together. 
Average embedding can hence use similar words to find similarity for text classification and can detect word semantic differences where countvectorizer is limited. 
2. The CountVectorizer approach is more interpretable since word representations in average embedding can be difficult to interpret in terms of similarity since it is based on frequency of counts.
3. Yes we are using transfer learning by using spacy's pre trained model. There is no significant difference in the testing scores in the classification report between the transfer learning's average embedding approach and CountVectorizer so we are not really observing the benefits (the f1 score is slightly lower in the embedding). 
However, looking at the training scores, the average embedding approach shows that the gap between train and test score is lower compared to countvectorizer indicating its much less overfit. 

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 2: Topic modeling 

The overarching goal of topic modeling is understanding high-level themes in a large collection of texts in an unsupervised way. 

In this exercise you will explore topics in a subset of `scikit-learn`'s [20 newsgroups text dataset](https://scikit-learn.org/0.19/datasets/twenty_newsgroups.html) using `scikit-learn`'s `LatentDirichletAllocation` (LDA) model. 

Usually, topic modeling is used for discovering abstract "topics" that occur in a collection of documents when you do not know the actual topics present in the documents. But 20 newsgroups text dataset is labeled with categories (e.g., sports, hardware, religion), and you will be able to cross-check the topics discovered by your model with these available topics. 

The starter code below loads the train and test portion of the data and convert the train portion into a pandas DataFrame. For speed, we will only consider documents with the following 8 categories. 

In [28]:
from sklearn.datasets import fetch_20newsgroups

In [29]:
cats = [
    "rec.sport.hockey",
    "rec.sport.baseball",
    "soc.religion.christian",
    "alt.atheism",
    "comp.graphics",
    "comp.windows.x",
    "talk.politics.mideast",
    "talk.politics.guns",
]  # We'll only consider these categories out of 20 categories for speed.

newsgroups_train = fetch_20newsgroups(
    subset="train", remove=("headers", "footers", "quotes"), categories=cats
)
X_news_train, y_news_train = newsgroups_train.data, newsgroups_train.target
df = pd.DataFrame(X_news_train, columns=["text"])
df["target"] = y_news_train
df["target_name"] = [
    newsgroups_train.target_names[target] for target in newsgroups_train.target
]
df

,text,target,target_name
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics
3,"The following problem is really bugging me,\na...",2,comp.windows.x
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast
...,...,...,...
4558,Hi Everyone ::\n\nI am looking for some soft...,1,comp.graphics
4559,Archive-name: x-faq/part3\nLast-modified: 1993...,2,comp.windows.x
4560,"\nThat's nice, but it doesn't answer the quest...",6,talk.politics.guns
4561,"Hi,\n I just got myself a Gateway 4DX-33V ...",2,comp.windows.x


In [30]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.windows.x',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast']

<br><br>

<!-- BEGIN QUESTION -->

### 2.1 Preprocessing using [spaCy](https://spacy.io/)
rubric={points:8}

In this exercise you'll prepare the data for topic modeling using [spaCy](https://spacy.io/). Preprocessing is a crucial step before training an LDA model and it markedly affects topic modeling results. So let's carry out preprocessing. 

**Your tasks:** 

Write code to carry out preprocessing of the "text" column in the dataframe above and store the preprocessed text in a new column called "text_pp" in the dataframe. 


Note that there is no such thing as "perfect" preprocessing. You'll have to make your own judgments and decisions on which tokens are more informative and which ones are less informative for the given task. Some common text preprocessing steps for topic modeling are: 
- getting rid of slashes or other weird characters
- sentence segmentation and tokenization      
- getting rid of urls and email addresses
- getting rid of other fairly unique tokens which are not going to help us in topic modeling  
- excluding stopwords and punctuation 
- lemmatization        

You might have to go back and forth between the preprocessing and topic modeling and interpretation steps in the next exercises. 

> Check out [these available attributes](https://spacy.io/api/token#attributes) for `token` in spaCy which might help you with preprocessing. 

> You can also get rid of words with specific POS tags. [Here](https://spacy.io/api/annotation/#pos-en) is the list of part-of-speech tags used in spaCy. 

> Note that preprocessing the corpus might take some time. So here are a couple of suggestions: 1) During the debugging phase, work on a smaller subset of the data. 2) Once you're done with the preprocessing part, you might want to save the preprocessed data so that you don't run the preprocessing part every time you run the notebook. 


<div class="alert alert-warning">

Solution_2_1
    
</div>

_Type your answer here, replacing this text._

In [31]:
def preprocess(
    doc,
    min_token_len=2,
    irrelevant_pos=["ADV", "PRON", "CCONJ", "PUNCT", "PART", "DET", "ADP", "SPACE", "SYN"],
):
    """
    Given text, min_token_len, and irrelevant_pos carry out preprocessing of the text
    and return a preprocessed string.

    Parameters
    -------------
    doc : (spaCy doc object)
        the spacy doc object of the text
    min_token_len : (int)
        min_token_length required
    irrelevant_pos : (list)
        a list of irrelevant pos tags

    Returns
    -------------
    (str) the preprocessed text
    """

    clean_text = []

    for token in doc:
        if (
            token.is_stop == False  # Check if it's not a stopword
            and len(token) > min_token_len  # Check if the word meets minimum threshold
            and token.pos_ not in irrelevant_pos # Check if the POS is in the acceptable POS tags
            and token.like_email == False # Check if it's not an email
            and token.like_url == False # Check if it's not an url
            and token.is_punct == False
            and token.is_ascii == True
            and token.like_num == False
        ):  
            lemma = token.lemma_  # Take the lemma of the word
            clean_text.append(lemma.lower().replace('\\', ''))
    return " ".join(clean_text)

In [32]:
df["text_pp"] = [preprocess(text) for text in nlp.pipe(df["text"])]

In [33]:
df

,text,target,target_name,text_pp
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns,know read u.s.c. sence wonder help u.s.c. prov...
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics,bad question ref list algorithm think bit hard...
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics,actuallay hand support idea have newsgroup asp...
3,"The following problem is really bugging me,\na...",2,comp.windows.x,following problem bug appreciate help create w...
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast,late upi foreign ministry spokesman ferhat ata...
...,...,...,...,...
4558,Hi Everyone ::\n\nI am looking for some soft...,1,comp.graphics,look software call shadow know simple raytrace...
4559,Archive-name: x-faq/part3\nLast-modified: 1993...,2,comp.windows.x,archive faq part3 modify 1993/04/04 subject pa...
4560,"\nThat's nice, but it doesn't answer the quest...",6,talk.politics.guns,nice answer question difference feds mandate l...
4561,"Hi,\n I just got myself a Gateway 4DX-33V ...",2,comp.windows.x,get gateway 4dx-33v try configure x11r5 need c...


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.2 Justification
rubric={points:2}

**Your tasks:**

Outline the preprocessing steps you carried out in the previous exercise and provide a brief justification for these steps. 

<div class="alert alert-warning">

Solution_2_2
    
</div>

- Removed stopwords because they appear frequently and contain little to no meaning
- Checked if the word is more than 2 characters long because any less and it is difficult to derive meaning
- Removed tokens that can be classified as certain part of speech due to the little amount of information they contain
- Removed tokens that were like emails and urls because they are, for the most part, unique
- Removed tokens that were just punctuation because it does not contain information
- Removed tokens containing special characters (not ascii) because they are relatively unique and would be difficult to use
- Removed numeric tokens due to how context related they are
- Got rid of '\'s inside tokens to correct typos

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.3 Build a topic model using sklearn's LatentDirichletAllocation
rubric={points:4}

**Your tasks:**
1. Create a topic model on the preprocessed data using [sklearn's `LatentDirichletAllocation`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html). Pick a reasonable number for `n_components`, i.e., number of topics and briefly justify your choice.

<div class="alert alert-warning">

Solution_2_3
    
</div>

In [34]:
from sklearn.decomposition import LatentDirichletAllocation

vec = CountVectorizer(stop_words='english')
X = vec.fit_transform(df["text_pp"])

In [35]:
n_topics = 9
lda = LatentDirichletAllocation(
    n_components=n_topics, learning_method="batch", max_iter=10, random_state=0
)
document_topics = lda.fit_transform(X)

We started by doing 10 categories and listing their words for each topic. We found that some topics were quite similar to each other so we narrowed it down to 8. Then some categories became too broad and or did not make too much sense. 9 was the perfect middleground between the two choices of n_topics we did that allowed us to construct labels for each topic.

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.4 Exploring word topic association
rubric={points:5}

**Your tasks:**
1. Show top 10 words for each of your topics and suggest labels for each of the topics (similar to how we came up with labels "health and nutrition", "fashion", and "machine learning" in the toy example we saw in class). 

> If your topics do not make much sense, you might have to go back to preprocessing in Exercise 2.1, improve it, and train your LDA model again. 

<div class="alert alert-warning">

Solution_2_4
    
</div>

In [36]:
sorting = np.argsort(lda.components_, axis=1)[:, ::-1]
feature_names = np.array(vec.get_feature_names_out())

In [37]:
import mglearn

mglearn.tools.print_topics(
    topics=range(n_topics),
    feature_names=feature_names,
    sorting=sorting,
    topics_per_chunk=5,
    n_words=10,
)

topic 0       topic 1       topic 2       topic 3       topic 4       
--------      --------      --------      --------      --------      
image         people        file          god           game          
window        say           entry         jesus         team          
file          know          output        believe       play          
use           come          program       church        year          
program       armenian      section       think         good          
available     time          build         christ        player        
server        think         char          faith         win           
run           armenians     line          christian     season        
widget        kill          use           know          think         
display       jews          rule          people        time          


topic 5       topic 6       topic 7       topic 8       
--------      --------      --------      --------      
gun           israel        team

Topic labels:

- 0: Computer display
- 1: Social movement
- 2: Computer programs
- 3: Christianity
- 4: Sports league
- 5: Gun politics
- 6: Middle east
- 7: Hockey
- 8: Debating/Philosophy

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.5 Exploring document topic association
rubric={points:5}

**Your tasks:**
1. Show the document topic assignment of the first five documents from `df`. 
2. Comment on the document topic assignment of the model. 

<div class="alert alert-warning">

Solution_2_5
    
</div>

In [38]:
for arr in document_topics[0:5]:
    print(np.argmax(arr), ":", arr)
df

# The document topics here are the 5 to doc1, 6 to doc2, 8 to doc3, 2 to doc4, 1 to doc4

5 : [0.00217959 0.27495059 0.00217907 0.00217962 0.00217929 0.37966823
 0.0021789  0.00217874 0.33230596]
6 : [0.30049596 0.00156569 0.00156561 0.00156553 0.00156579 0.0015655
 0.55120118 0.00156517 0.13890958]
8 : [0.29983867 0.00202131 0.00202057 0.00202117 0.00202086 0.00202072
 0.10476911 0.00202106 0.58326654]
2 : [0.33871892 0.00383594 0.63444629 0.00383526 0.00383227 0.00383194
 0.00383185 0.0038319  0.00383563]
1 : [0.00258463 0.97932194 0.00258441 0.00258488 0.00258448 0.0025845
 0.00258548 0.00258493 0.00258476]


,text,target,target_name,text_pp
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns,know read u.s.c. sence wonder help u.s.c. prov...
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics,bad question ref list algorithm think bit hard...
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics,actuallay hand support idea have newsgroup asp...
3,"The following problem is really bugging me,\na...",2,comp.windows.x,following problem bug appreciate help create w...
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast,late upi foreign ministry spokesman ferhat ata...
...,...,...,...,...
4558,Hi Everyone ::\n\nI am looking for some soft...,1,comp.graphics,look software call shadow know simple raytrace...
4559,Archive-name: x-faq/part3\nLast-modified: 1993...,2,comp.windows.x,archive faq part3 modify 1993/04/04 subject pa...
4560,"\nThat's nice, but it doesn't answer the quest...",6,talk.politics.guns,nice answer question difference feds mandate l...
4561,"Hi,\n I just got myself a Gateway 4DX-33V ...",2,comp.windows.x,get gateway 4dx-33v try configure x11r5 need c...


While our model did not correctly predict all of our target categories from the df, it does make sense how our model did make predictions. Looking at the key words of the first 5 examples, we can see why our model predicts what it does. For example, it makes sense how the first example is predicted as "gun politics" as it contains a lot of key words about law. Additionally, example 3 also has words such as "interested", "agree", and "think", which results in the model mainly classifying it as "Debating/Philosophy" while also having some percentage in "Computer display".



In [39]:
print(df["text_pp"][0])
print(df["text_pp"][2])

know read u.s.c. sence wonder help u.s.c. provide paragraph shall unlawful person transfer possess machinegun get look law dictionary find person artificial entity create government right federal constitution understand statute enforce individual tell government tell possess pass law law know law constitional go court go court run mill guilty jail claim right possess tell supreme court right
actuallay hand support idea have newsgroup aspect graphic programming propose brian reply original posting suggest loose structure comp.graphics.hw_and_sw reason make group possible reason spilt posting day take time browse posting find interested understand agree want aspect graphic meeting agree extension news forum exchange idea help help think difficult achive different thing meeting good evening netters|-


<!-- BEGIN QUESTION -->

## Exercise 3: Short answer questions 
<hr>

rubric={points:6}

1. In lecture 19, we talked about multi-class classification. Comment on how each model in the list below might be handling multiclass classification. Check `scikit-learn` documentation for each of these models when you answer this question.  
    - Decision Tree
    - KNN
    - Random Forest    
    - Logistic Regression
    - SVM RBF
2. What is transfer learning in natural language processing or computer vision? Briefly explain.     
3. In Lecture 19 we briefly discussed how neural networks are sort of like `Pipeline`s, in the sense that they involve multiple sequential transformations of the data, finally resulting in the prediction. Why was this property useful when it came to transfer learning?

<div class="alert alert-warning">

Solution_3
    
</div>

### Question 1

- Decision Tree - It supports multiclass classification without needing to build binary models. At each split, it makes a decision on which class (out of all the classes) should be the true or false case, maximizing its criterion.
- kNN - Supports multiclass classification without needing to build binary models. Simply stores the X and y and finds the vote of the k-nearest neighbors in prediction allowing for muticlass classificaiton. 
- Random Forest: The number of outputs will be single or multiple depending on the target method of classification (binary or multi), and it can be checked by accessing the n_classes/classes attributes.
- Logistic Regression - For default selection, if the data is binary, it selects One-vs-Rest otherwise it selects "multinomial" i.e. it tries to reduce loss over entire probability distribution. 
- SVM RBF - It uses One-vs-Rest approach as default

### Question 2 


In transfer learning, a pre-trained model is repurposed for our task. Usually most of the layers have their hyperparameters optimized which makes it simpler to deploy in a new similar task. 

There are three common ways to use transfer learning in computer vision:

- Using pre-trained models out-of-the-box
- Using pre-trained models as feature extractor and training your own model with these features
- Starting with weights of pre-trained models and fine-tuning the weights for your task.

For example, in NLP using pre-trained embeddings from spaCy we can perform document classifications or doing image classifications in computer vision.

### Question 3

Using a transfer learning neural network, latent features were easily extracted without having to optimize complicated hyperparameters. The resulting features were then easily used to train our model in lecture to give output for specific classification of images according to the task. This saves time and resources to optimize hyperparameters and hence makes ML approaches more easy to use.

<!-- END QUESTION -->

<br><br><br><br>

**PLEASE READ BEFORE YOU SUBMIT:** 

When you are ready to submit your assignment do the following:

1. Run all cells in your notebook to make sure there are no errors by doing `Kernel -> Restart Kernel and Clear All Outputs` and then `Run -> Run All Cells`. 
2. Notebooks with cell execution numbers out of order or not starting from "1" will have marks deducted. Notebooks without the output displayed may not be graded at all (because we need to see the output in order to grade your work).
3. Upload the assignment using Gradescope's drag and drop tool. Check out this [Gradescope Student Guide](https://lthub.ubc.ca/guides/gradescope-student-guide/) if you need help with Gradescope submission. 
4. Make sure that the plots and output are rendered properly in your submitted file. If the .ipynb file is too big and doesn't render on Gradescope, also upload a pdf or html in addition to the .ipynb so that the TAs can view your submission on Gradescope. 

![](img/eva-well-done.png)